In [1]:
import spacy
import pandas as pd

In [2]:
# Load spacy NLP model
nlp = spacy.load("en_core_web_lg", disable=["ner", "textcat"])

In [3]:
def word2features(sent, i):
    

        

    features = {
        "bias": 1.0,
        "lemma": sent[i].lemma_,
        #"pos": sent[i].pos_,
        #"tag": sent[i].tag_,
        #"dep": sent[i].dep_,
        "shape": sent[i].shape_,
        "is_alpha": sent[i].is_alpha,
        "is_stop": sent[i].is_stop,
        "is_title": sent[i].is_title,
        "like_num": sent[i].like_num,
        "is_punct": sent[i].is_punct,
        "is_left_punct": sent[i].is_left_punct,
        "is_right_punct": sent[i].is_right_punct,
        
    }
    if i > 0:
        features.update(
            {
                "-1:lemma": sent[i - 1].lemma_,
                #"-1:pos": sent[i - 1].pos_,
                #"-1:tag": sent[i - 1].tag_,
                #"-1:dep": sent[i - 1].dep_,
                "-1:shape": sent[i - 1].shape_,
                "-1:is_alpha": sent[i - 1].is_alpha,
                "-1:is_stop": sent[i - 1].is_stop,
                "-1:is_title": sent[i - 1].is_title,
                "-1:like_num": sent[i - 1].like_num,
                "-1:is_punct": sent[i - 1].is_punct,
                "-1:is_left_punct": sent[i - 1].is_left_punct,
                "-1:is_right_punct": sent[i - 1].is_right_punct,
                "after_or": any([x.lemma_ == "or" for x in sent[:i]]),
                "parens": any([x.lemma_ == "(" for x in sent[:i]]) and not any([x.lemma_ == ")" for x in sent[:i]])
            }
        )
        if i > 1:
            features.update(
                {
                    "-2:lemma": sent[i - 2].lemma_,
                    #"-2:pos": sent[i - 2].pos_,
                    #"-2:tag": sent[i - 2].tag_,
                    #"-2:dep": sent[i - 2].dep_,
                    "-2:shape": sent[i - 2].shape_,
                    "-2:is_alpha": sent[i - 2].is_alpha,
                    "-2:is_stop": sent[i - 2].is_stop,
                    "-2:is_title": sent[i - 2].is_title,
                    "-2:like_num": sent[i - 2].like_num,
                    "-2:is_punct": sent[i - 2].is_punct,
                    "-2:is_left_punct": sent[i - 2].is_left_punct,
                    "-2:is_right_punct": sent[i - 2].is_right_punct,
                    "-1:after_or": any([x.lemma_ == "or" for x in sent[:i-1]]),
                    "-1:parens": any([x.lemma_ == "(" for x in sent[:i-1]]) and not any([x.lemma_ == ")" for x in sent[:i-1]])
                }
            )
            """if i > 2:
                features.update(
                    {
                        "-3:lemma": sent[i - 3].lemma_,
                        #"-2:pos": sent[i - 2].pos_,
                        #"-2:tag": sent[i - 2].tag_,
                        #"-2:dep": sent[i - 2].dep_,
                        "-3:shape": sent[i - 3].shape_,
                        "-3:is_alpha": sent[i - 3].is_alpha,
                        "-3:is_stop": sent[i - 3].is_stop,
                        "-3:is_title": sent[i - 3].is_title,
                        "-3:like_num": sent[i - 3].like_num,
                        "-3:is_punct": sent[i - 3].is_punct,
                        "-3:is_left_punct": sent[i - 3].is_left_punct,
                        "-3:is_right_punct": sent[i - 3].is_right_punct,
                    }
                )"""
    else:
        features["BOS"] = True

    if i < len(sent) - 1:
        features.update(
            {
                "+1:lemma": sent[i + 1].lemma_,
                #"+1:pos": sent[i + 1].pos_,
                #"+1:tag": sent[i + 1].tag_,
                #"+1:dep": sent[i + 1].dep_,
                "+1:shape": sent[i + 1].shape_,
                "+1:is_alpha": sent[i + 1].is_alpha,
                "+1:is_stop": sent[i + 1].is_stop,
                "+1:is_title": sent[i + 1].is_title,
                "+1:like_num": sent[i + 1].like_num,
                "+1:is_punct": sent[i + 1].is_punct,
                "+1:is_left_punct": sent[i + 1].is_left_punct,
                "+1:is_right_punct": sent[i + 1].is_right_punct,
                "+1:after_or": any([x.lemma_ == "or" for x in sent[:i+1]]),
                "+1:parens": any([x.lemma_ == "(" for x in sent[:i+1]]) and not any([x.lemma_ == ")" for x in sent[:i+1]])
            }
        )
        if i < len(sent) - 2:
            features.update(
                {
                    "+2:lemma": sent[i + 2].lemma_,
                    #"+2:pos": sent[i + 2].pos_,
                    #"+2:tag": sent[i + 2].tag_,
                    #"+2:dep": sent[i + 2].dep_,
                    "+2:shape": sent[i + 2].shape_,
                    "+2:is_alpha": sent[i + 2].is_alpha,
                    "+2:is_stop": sent[i + 2].is_stop,
                    "+2:is_title": sent[i + 2].is_title,
                    "+2:like_num": sent[i + 2].like_num,
                    "+2:is_punct": sent[i + 2].is_punct,
                    "+2:is_right_punct": sent[i + 2].is_left_punct,
                    "+2:is_right_punct": sent[i + 2].is_right_punct,
                    "+2:after_or": any([x.lemma_ == "or" for x in sent[:i+2]]),
                    "+2:parens": any([x.lemma_ == "(" for x in sent[:i+2]]) and not any([x.lemma_ == ")" for x in sent[:i+2]])
                }
            )
            """if i < len(sent) - 3:
                features.update(
                    {
                        "+3:lemma": sent[i + 3].lemma_,
                        #"+2:pos": sent[i + 2].pos_,
                        #"+2:tag": sent[i + 2].tag_,
                        #"+2:dep": sent[i + 2].dep_,
                        "+3:shape": sent[i + 2].shape_,
                        "+3:is_alpha": sent[i + 3].is_alpha,
                        "+3:is_stop": sent[i + 3].is_stop,
                        "+3:is_title": sent[i + 3].is_title,
                        "+3:like_num": sent[i + 3].like_num,
                        "+3:is_punct": sent[i + 3].is_punct,
                        "+3:is_right_punct": sent[i + 3].is_left_punct,
                        "+3:is_right_punct": sent[i + 3].is_right_punct,
                    }
                )"""
    else:
        features["EOS"] = True

    return features

In [4]:
# Load cleaned data
training_data = pd.read_pickle("../data/interim/crf_data.pickle")
#test_data = pd.read_pickle("../data/interim/crf_test_data.pickle")

In [5]:
training_data.head(25)

,input,name,qty,unit
index,,,,
0,1.25 cups cooked and pureed fresh butternut sq...,butternut squash,1.25,cup
1,1 cup peeled and cooked fresh chestnuts (about...,chestnut,1,cup
2,"1 medium size onion, peeled and chopped",onion,1,
3,"2 stalks celery, chopped coarse",celery,2,stalk
4,1.5 tablespoons vegetable oil,vegetable oil,1.5,tablespoon
6,"2 tablespoons unflavored gelatin, dissolved in...",gelatin,2,tablespoon
7,Salt,Salt,NaN,
8,1 cup canned plum tomatoes with juice,plum tomato,1,cup
9,6 cups veal or beef stock,veal stock,6,cup


In [6]:

# have spacy parse the input string with the full pipeline to generate features this will take some time
training_data["input"] = list(nlp.pipe(training_data["input"].astype('unicode').values, batch_size=50))

#test_data["input"] = list(nlp.pipe(test_data["input"].astype('unicode').values, batch_size=50))


In [7]:
training_data.head(25)

,input,name,qty,unit
index,,,,
0,"(1.25, cups, cooked, and, pureed, fresh, butte...",butternut squash,1.25,cup
1,"(1, cup, peeled, and, cooked, fresh, chestnuts...",chestnut,1,cup
2,"(1, medium, size, onion, ,, peeled, and, chopped)",onion,1,
3,"(2, stalks, celery, ,, chopped, coarse)",celery,2,stalk
4,"(1.5, tablespoons, vegetable, oil)",vegetable oil,1.5,tablespoon
6,"(2, tablespoons, unflavored, gelatin, ,, disso...",gelatin,2,tablespoon
7,(Salt),Salt,NaN,
8,"(1, cup, canned, plum, tomatoes, with, juice)",plum tomato,1,cup
9,"(6, cups, veal, or, beef, stock)",veal stock,6,cup


In [8]:
crf_training_features = training_data["input"].apply(lambda doc: [word2features(doc,i) for i in range(len(doc))])
                                                  
#crf_test_features = test_data["input"].apply(lambda doc: [word2features(doc,i) for i in range(len(doc))])                                                  
#training_data.iloc[0]["input"].apply(lambda doc: [word2features(doc,i) for i in range(len(doc))])
#[[word2features(training_data.iloc[1]["input"],i) for i in range(len(training_data.iloc[0]["input"]))]]


In [9]:
crf_training_features[0]

[{'bias': 1.0,
  'lemma': '1.25',
  'shape': 'd.dd',
  'is_alpha': False,
  'is_stop': False,
  'is_title': False,
  'like_num': True,
  'is_punct': False,
  'is_left_punct': False,
  'is_right_punct': False,
  'BOS': True,
  '+1:lemma': 'cup',
  '+1:shape': 'xxxx',
  '+1:is_alpha': True,
  '+1:is_stop': False,
  '+1:is_title': False,
  '+1:like_num': False,
  '+1:is_punct': False,
  '+1:is_left_punct': False,
  '+1:is_right_punct': False,
  '+1:after_or': False,
  '+1:parens': False,
  '+2:lemma': 'cook',
  '+2:shape': 'xxxx',
  '+2:is_alpha': True,
  '+2:is_stop': False,
  '+2:is_title': False,
  '+2:like_num': False,
  '+2:is_punct': False,
  '+2:is_right_punct': False,
  '+2:after_or': False,
  '+2:parens': False},
 {'bias': 1.0,
  'lemma': 'cup',
  'shape': 'xxxx',
  'is_alpha': True,
  'is_stop': False,
  'is_title': False,
  'like_num': False,
  'is_punct': False,
  'is_left_punct': False,
  'is_right_punct': False,
  '-1:lemma': '1.25',
  '-1:shape': 'd.dd',
  '-1:is_alpha': Fa

In [10]:
crf_training_features.to_pickle("../data/interim/crf_training_features.pickle")
#crf_test_features.to_pickle("../data/interim/crf_test_features.pickle")